In [ ]:
%cd /home/smalani/Cybernetic/growthcurvefit/
import pandas as pd

In [ ]:
df = pd.read_csv('/home/smalani/Cybernetic/growthcurvefit/data/Data_For_Importation.csv')
df.head()

df['M0'] = df['M0'] * 10 / 6
df['M1'] = df['M1'] * 10 / 6
df['M2'] = df['M2'] * 10 / 6
df['M3'] = df['M3'] * 10 / 6
df['M4'] = df['M4'] * 10 / 6
df['M5'] = df['M5'] * 10 / 6
df['M6'] = df['M6'] * 10 / 6
df['M7'] = df['M7'] * 10 / 6


In [ ]:
df_par = pd.read_csv('/home/smalani/Cybernetic/growthcurvefit/data/LightParameter.csv')
print(df_par.head())
print(df_par)

df_dict = df_par.to_dict('list')


In [ ]:
df_par.nunique(axis=1)[0]

In [ ]:
def growthfun(t,x,p):

    # Parameters
    mu_max, Ks, Y = p

    # Variables
    S, X = x
    
    
    mu = mu_max * S / (Ks + S)

    dSdt = - (mu / Y) * X
    dXdt = mu * X

    return [dSdt, dXdt]

def baranyifun(t_arr, p):

    # Parameters
    mu_max, x0, xmax, lambd = p

    y0 = np.log(x0)
    ymax = np.log(xmax)

    A = t_arr + (1/mu_max) * np.log(np.exp(-mu_max * t_arr) + np.exp(-mu_max * lambd) - np.exp(-mu_max * t_arr -mu_max * lambd))

    y_t = y0 + mu_max * A - np.log(1 + (np.exp(mu_max * A) - 1) / (np.exp(ymax - y0)))
    x_t = np.exp(y_t)

    return x_t

In [ ]:
import matplotlib.pyplot as plt



fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.plot(df['Time'], df['M0'], '.', label='M0')

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.semilogy(df['Time'], df['M0'], '.', label='M0')

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp

def fsolvefun(pars):
    mu_max, xmax, lambd = pars

    x0 = 0.01
    pars = mu_max, x0, xmax, lambd
    t_arr = df['Time']
    x_t = baranyifun(t_arr, pars)
    
    return np.mean((x_t - df['M0']) ** 2) #+ Ks * 0.01

In [ ]:
print(df['Time'].iloc[-1])

In [ ]:
from scipy.optimize import minimize

fsolve_guess = [0.3, 10, 0]
fsolve_sol = minimize(fsolvefun, fsolve_guess, method='L-BFGS-B', bounds=((1e-10, None), (1e-10, None), (0, None)), options={'eps': 1e-11})

print(fsolve_sol)


In [ ]:
import matplotlib.pyplot as plt

xinit = [2, 0.01]
x0 = 0.01

fsolve_guess = [0.3, 10, 0]

mu_max, ymax, lambd = fsolve_sol.x
# mu_max, ymax, lambd = fsolve_guess
pars = mu_max, x0, ymax, lambd

t_arr = df['Time']
x_t = baranyifun(t_arr, pars)


fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)
ax.plot(df['Time'], df['M0'], '.', label='M0')
ax.plot(t_arr, x_t, label='Fit')

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp

# def fsolvefun(pars):
#     mu_max, xmax, lambd = pars

#     x0 = 0.01
#     pars = mu_max, x0, xmax, lambd
#     t_arr = df['Time']
#     x_t = baranyifun(t_arr, pars)
    
#     return np.mean((x_t - df['M0']) ** 2) #+ Ks * 0.01

def fsolvefun(pars):
    print(pars)
    mu_maxs = pars[:8]
    lambdas = pars[8:16]
    xmax = pars[16]
    x0 = 0.01

    error = 0
    for i in range(len(lambdas)):
        lambd = lambdas[i]
        mu_max = mu_maxs[i]
        t_arr = df['Time']
        ars = mu_max, x0, xmax, lambd
        x_t = baranyifun(t_arr, [mu_max, x0, xmax, lambd])
        reactor = 'M' + str(i)
        error += np.mean((x_t - df[reactor]) ** 2)

    return error

In [ ]:
fsolve_guess = [0.3, 0.3, 0.3, 0.3, 0.2, 0.2, 0.1, 0.1,
                 1, 1, 1, 1, 1, 1, 1, 1,
                  10]
bounds = tuple([(1e-10, None)] * 16 + [(1e-10, None)])
fsolve_sol = minimize(fsolvefun, fsolve_guess, method='L-BFGS-B', bounds=bounds, options={'eps': 1e-11})
print(fsolve_sol)

In [ ]:
import matplotlib.pyplot as plt

lambdas = fsolve_sol.x[:8]
mu_maxs = fsolve_sol.x[8:16]
xmax = fsolve_sol.x[16]
x0 = 0.01



error = 0
for i in range(len(lambdas)):
    lambd = lambdas[i]
    mu_max = mu_maxs[i]
    t_arr = df['Time']
    ars = mu_max, x0, xmax, lambd
    x_t = baranyifun(t_arr, [mu_max, x0, xmax, lambd])
    error += np.mean((x_t - df['M' + str(i)]) ** 2)